This script builds a model for classifying two products from different retailers as being the same or not, based on data from the retailers. In currently uses the following features:
- Cosine distance between:
    - Product names
    - Product descriptions

The last two cells find the inter-retailer price differences for products known to be the same and determine the value of correctly idenfitied arbitrage opportunities
    

In [113]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

In [194]:
# Load the data
amazon = pd.read_csv("amazon.csv", na_filter=False, encoding="ISO-8859-1")
google = pd.read_csv("googleproducts.csv", na_filter=False, encoding="ISO-8859-1")
amazon_google = pd.read_csv("Amzon_GoogleProducts_perfectMapping.csv", encoding="ISO-8859-1")
amazon_google

,idAmazon,idGoogleBase
0,b000jz4hqo,http://www.google.com/base/feeds/snippets/1844...
1,b00004tkvy,http://www.google.com/base/feeds/snippets/1844...
2,b000g80lqo,http://www.google.com/base/feeds/snippets/1844...
3,b0006se5bq,http://www.google.com/base/feeds/snippets/1842...
4,b00021xhzw,http://www.google.com/base/feeds/snippets/1843...
5,b0000dbykm,http://www.google.com/base/feeds/snippets/1843...
6,b0000dbykm,http://www.google.com/base/feeds/snippets/1836...
7,b00029bqa2,http://www.google.com/base/feeds/snippets/1843...
8,b0007prnjo,http://www.google.com/base/feeds/snippets/1843...
9,b000i82j80,http://www.google.com/base/feeds/snippets/1843...


In [135]:
# create a table with all the matched products
data = pd.merge(amazon, amazon_google, left_on="id", right_on="idAmazon")
data = pd.merge(data, google, left_on="idGoogleBase", right_on="id")
data = data[['title', 'description_x', 'name', 'description_y']]
data.columns = ['amazon_name', 'amazon_description', 'google_name', 'google_description']
data['match'] = np.ones(data.shape[0])
data    

,idAmazon,idGoogleBase
0,b000jz4hqo,http://www.google.com/base/feeds/snippets/1020...
1,b00004tkvy,http://www.google.com/base/feeds/snippets/8112...
2,b000g80lqo,http://www.google.com/base/feeds/snippets/5733...
3,b0006se5bq,http://www.google.com/base/feeds/snippets/1730...
4,b00021xhzw,http://www.google.com/base/feeds/snippets/1838...
5,b0000dbykm,http://www.google.com/base/feeds/snippets/1745...
6,b0000dbykm,http://www.google.com/base/feeds/snippets/1754...
7,b00029bqa2,http://www.google.com/base/feeds/snippets/1211...
8,b0007prnjo,http://www.google.com/base/feeds/snippets/2263...
9,b000i82j80,http://www.google.com/base/feeds/snippets/1264...


In [ ]:
# create table of mismatches
def findDifferent(amazonId):
    googleMatches = amazon_google.loc[amazon_google.idAmazon==amazonId].idGoogleBase
    googleNotMatches = google.loc[google.id.isin(googleMatches)==False]
    googleNotMatches = googleNotMatches.iloc[np.random.permutation(len(googleNotMatches) - 1)]
    return googleNotMatches[:1].iloc[0].id

amazon_not_google = pd.DataFrame(amazon_google.idAmazon)
amazon_not_google['idGoogleBase'] = amazon_not_google.idAmazon.map(findDifferent)
amazon_not_google

In [136]:
# add mismatched examples to matched ones
moreData = pd.merge(amazon, amazon_not_google, left_on="id", right_on="idAmazon")
moreData = pd.merge(moreData, google, left_on="idGoogleBase", right_on="id")
moreData = moreData[['title', 'description_x', 'name', 'description_y']]
moreData.columns = ['amazon_name', 'amazon_description', 'google_name', 'google_description']
moreData['match'] = np.zeros(moreData.shape[0])
data = pd.concat([data, moreData])
data

,amazon_name,amazon_description,google_name,google_description,match
0,clickart 950 000 - premier image pack (dvd-rom),,clickart 950000 - premier image pack (dvd-rom),massive collection of images & fonts for all y...,1
1,noah's ark activity center (jewel case ages 3-8),,the beginners bible: noah's ark activity cente...,,1
2,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage (ptree) - vernfp2007rt - premium accounti...,if you're like most nonprofit organizations yo...,1
3,singing coach unlimited,singing coach unlimited - electronic learning ...,singing coach unlimited - electronic learning ...,learn to sing with the help of a patented real...,1
4,adobe after effects professional 6.5 upgrade f...,upgrade only; installation of after effects st...,adobe software 22070152 after effects 6.5 pbupgrd,adobe after effects pb 6.5 win upgrade.standar...,1
5,mia's math adventure: just in time,in mia's math adventure: just in time children...,kutoka interactive 61208 mia's math adventure:...,mia's math adventure tells a captivating story...,1
6,mia's math adventure: just in time,in mia's math adventure: just in time children...,kutoka interactive 61208 - mias math adventure...,kutoka interactive 61208 : mia s math adventur...,1
7,disney's 1st & 2nd grade bundle (pixar 1st gra...,disney's 1st & 2nd grade bundle will help your...,disney learning 1st-2nd grade win,disney learning 1st & 2nd features an all-star...,1
8,paper art: gift wrapping,how many times have you heard that it's the th...,paper art volume 2: gift wrapping,how many times have you heard that it's the th...,1
9,spy sweeper spanish,,webroot software inc - 31250 - spy sweeper spa...,a sypware infection is no longer just a matter...,1


In [140]:
# create distance features
def cosDist(s1, s2):
    returnDist = 1
    try:
        vecs = TfidfVectorizer().fit_transform([s1,s2]).todense()
        cosDist = cosine(vecs[0], vecs[1])
        if not np.isnan(cosDist):
            returnDist = cosDist 
    except:
        pass
    return returnDist

data["descriptionDistance"] = list(map(lambda x: cosDist(x[0],x[1]), zip(data.google_description, data.amazon_description)))
data["nameDistance"] = list(map(lambda x: cosDist(x[0],x[1]), zip(data.google_name, data.amazon_name)))

,amazon_name,amazon_description,google_name,google_description,match,descriptionDistance,nameDistance
1034,cakewalk sonar 6 producer ed,sonar 6 producer is the definitive choice for ...,cakewalk sonar 6 producers edition software mu...,sonar 6 producers edition - multi-track audio/...,1,0.769739,8.056857e-01
271,sony acid music studio,acid music studio 6 was designed for the serio...,acid music studio 6,make your music more professional and more pow...,1,0.796791,2.234855e-01
1241,namo webeditor 2006,namo webeditor 2006 suite is one of the most c...,freeverse software toysight gold,toysight is set of cool games and toys to play...,0,0.670588,1.000000e+00
96,scrabble,play against one of eight challenging computer...,destineer inc 10328 scrabble mac 10.3.8 or later,play against one of eight challenging computer...,1,0.141743,7.403020e-01
784,canopus 77010158100 let's edit,canopus (770-10158-100) let's edit 2.0,sos aggregation company vw - gti racer - topwa...,gti racer offers the ability to race in new an...,0,1.000000,1.000000e+00
349,instant landscaping 3.0 [lb],instant landscape design 3.0 is the perfect to...,print explosion deluxe 3.0: mac,whether you're new to print media or a seasone...,0,0.948874,1.000000e+00
937,pitstop pro 7.0 upgr - 6.x or higher,pitstop pro 7.0 upgr - 6.x or higher,enfocus software pp7.0-ug-001 - pitstop pro 7....,enfocus software pp7.0-ug-001 : enfocus softwa...,1,0.664824,4.202613e-01
310,adobe premiere elements 3.0,adobe premiere elements 3.0 software makes cre...,adobe premiere elements 3 (pc),key features: create and share videos import v...,1,0.685031,2.234855e-01
804,handmark oxford american desk dictionary and t...,the oxford american desk dictionary and thesau...,pearson education book: the adobe photoshop li...,book: the adobe photoshop lightroom book: the ...,0,0.920297,1.000000e+00
22,icopydvds2 ultra by digital wunders,,29270077 adobe creative suite 3 web standard -...,adobe systems 29270077 : usually ships in 24 h...,0,1.000000,1.000000e+00


In [ ]:
# split data for cross validation
data = data.iloc[np.random.permutation(data.shape[0])]
msk = np.random.rand(data.shape[0]) < .8
train = data[msk]
test = data[~msk]
trainData, trainLabels = train[['nameDistance', 'descriptionDistance']], train['match']
testData, testLabels = test[['nameDistance', 'descriptionDistance']], test['match']

In [176]:
# fit and test a model
clf = RandomForestClassifier(100)
clf.fit(trainData, trainLabels)
preds = clf.predict(testData)
print("accuracy:\n" + str(np.mean(preds==testLabels)))
right = test.loc[preds==testLabels]
truePositives = right[right.match==1]
wrong = test.loc[preds!=testLabels]
falsePositives = wrong[wrong.match==0]
print("\nfalse positive count:\n" + str(falsePositives.shape[0]))
print("\nfalse positives")
falsePositives

accuracy:
0.981096408318

false positive count:
8

false positives


,amazon_name,amazon_description,google_name,google_description,match,descriptionDistance,nameDistance
1170,money home & business 2007 win32 eng na mini b...,model- ms-cd19383wi vendor- microsoft corporat...,myob accounting plus/accountedge additional u,designed to help get your business get up and ...,0,0.606781,1.000000
1177,the movies (mac),the movies answers a question that many of us ...,adobe illustrator cs3 (mac),key features: live color live trace live paint...,0,1.000000,0.829224
275,student home learning system for pc/mac,the fogware student home learning system is on...,pxl smartscale for mac/win,one image any size.,0,0.970236,0.793863
501,lego star wars ii: the original trilogy for mac,lego star wars ii sets players loose in the or...,novell identity manager integration module for...,mod3 mainframe lics+1yr mnt vlal1,0,1.000000,0.895448
797,acad corel painter x pc/mac,- marketing information: painter x the worldan...,premiere pro cs3 mac,adobe premiere pro cs3 mac. make every frame c...,0,0.751229,0.872640
865,adobe premiere pro cs3,tell your story with maximum impact using adob...,adobe(r) photoshop(r) 7.0 upgrade for pc,with versatility of its tools the adobe photos...,0,0.763237,0.872640
551,adobe premiere pro cs3,tell your story with maximum impact using adob...,adobe photoshop cs3 extended - complete packag...,ideal for film video and multimedia profession...,0,0.614225,0.805686
1126,spring cleaning 9.0 for mac os 10.4 & above,spring cleaning 9 secures your privacy and cle...,adobe after effects cs3 professional software ...,after effects cs3 professional software for ma...,0,0.811649,0.885532


In [158]:
# find price differences for matched products

priceData = pd.merge(amazon, amazon_google, left_on="id", right_on="idAmazon")
priceData = pd.merge(priceData, google, left_on="idGoogleBase", right_on="id")
priceData = priceData[['title', 'idAmazon', 'price_x', 'name',  'idGoogleBase', 'price_y']]
priceData.columns = ['amazon_name', 'amazon_id'
                     , 'amazon_price', 'google_name', 'google_id', 'google_price']

def floatMe(s):
    try:
        return float(s)
    except:
        return 0
    
priceData['amazon_price'] = priceData.amazon_price.map(floatMe)
priceData['google_price'] = priceData.google_price.map(floatMe)

priceData = priceData.loc[priceData.amazon_price!=0]
priceData = priceData.loc[priceData.google_price!=0]

priceData['priceDiff'] = priceData.amazon_price - priceData.google_price
priceData = priceData.sort('priceDiff')
priceData

,amazon_name,amazon_id,amazon_price,google_name,google_id,google_price,priceDiff
114,watchguard serverlock manager (100 servers),b00005atxo,14995.00,serverlock manager - 100 servers,http://www.google.com/base/feeds/snippets/1143...,56543.88,-41548.88
456,steinberg halion symphonic orchestra virtual i...,b000k6b2l0,374.99,steinberg software halion symphonic orchestra ...,http://www.google.com/base/feeds/snippets/1783...,545.51,-170.52
575,adobe photoshop cs2 (mac) [old version],b00081i6ji,649.00,adobe systems inc 13102124 adobe photoshop cs ...,http://www.google.com/base/feeds/snippets/1841...,788.63,-139.63
494,microsoft windows terminal server 2003 client ...,b00008mnxz,669.00,win 2003 ter svr cal 5pk microsoft r19-00846,http://www.google.com/base/feeds/snippets/1838...,762.95,-93.95
425,adobe creative suite cs3 design premium [mac],b000ndibri,1799.00,adobe cs3 design premium,http://www.google.com/base/feeds/snippets/2224...,1865.99,-66.99
1009,aquazone bass edition,b000gaqlxe,19.99,allume checkit system performance suite,http://www.google.com/base/feeds/snippets/1382...,84.99,-65.00
321,serious magic ultra2 master sets library 2 ( w...,b0009yx9by,276.50,serious magic ultra master sets library 1 effe...,http://www.google.com/base/feeds/snippets/5238...,339.95,-63.45
388,adobe creative suite cs3 web premium [mac],b000ndibvo,1599.00,adobe cs3 web premium,http://www.google.com/base/feeds/snippets/5767...,1659.99,-60.99
1011,adobe creative suite cs3 master collection upsell,b000ndibl4,1999.00,adobe cs3 master collection upsell from cs1,http://www.google.com/base/feeds/snippets/1499...,2054.99,-55.99
742,adobe creative suite cs3 production premium,b000ogsyjy,1699.00,adobe cs3 production premium,http://www.google.com/base/feeds/snippets/1657...,1749.99,-50.99


In [196]:
# calculate the potential value of arbitrage opportunities discovered in the test data
truePositivePrices = pd.merge(truePositives, priceData)
truePositivePrices['absPriceDiff'] = truePositivePrices.priceDiff.map(np.absolute)
print("Number of test examples:\n" + str(test.shape[0]))
print("Net Price Differences in Correctly Matched Products:\n" + str(truePositivePrices.absPriceDiff.sum()))
print("Net Price Differences under $2500 in Matched Products:\n" 
      + str(truePositivePrices.loc[truePositivePrices.absPriceDiff < 2500].absPriceDiff.sum()))
print("Net Price Differences under $500 in Matched Products:\n" 
      + str(truePositivePrices.loc[truePositivePrices.absPriceDiff < 500].absPriceDiff.sum()))
truePositivePrices.sort('absPriceDiff', 0, False)

Number of test examples:
529
Net Price Differences in Matched Products:
55029.47
Net Price Differences under $2500 in Matched Products:
8934.48
Net Price Differences under $500 in Matched Products:
5204.11


,amazon_name,amazon_description,google_name,google_description,match,descriptionDistance,nameDistance,amazon_id,amazon_price,google_id,google_price,priceDiff,absPriceDiff
205,microsoft licenses sps extrnlconnnonemplyengla...,microsoft sps external connector non emply eng...,microsoft h32-00034 sps extrnlconnnonemplyengl...,microsoft h32-00034 sps extrnlconnnonemplyengl...,1,0.896284,0.774235,b0006g2zce,101515.55,http://www.google.com/base/feeds/snippets/1682...,55420.56,46094.99,46094.99
168,espro + interpretair s/w,- marketing information: interpretair provides...,fluke networks intair-lap - interpretair wlan ...,fluke networks intair-lap : interpretair provi...,1,0.503422,0.913052,b000bi7uqs,5750.60,http://www.google.com/base/feeds/snippets/1840...,3277.97,2472.63,2472.63
83,flash remoting 1 alp ret eng cd 1u,- marketing information: macromedia flash remo...,adobe systems inc flash remoting 1.0 (net&java...,macromedia flash remoting mx provides the conn...,1,0.397323,0.844071,b000cs3s2m,1657.06,http://www.google.com/base/feeds/snippets/1750...,964.22,692.84,692.84
184,adobe framemaker 7.2 (windows),adobe framemaker 7.2 software delivers an ente...,adobe systems 27910468 adobe framemaker v.7.2 ...,free shipping available adobe systems 27910468...,1,0.968247,0.783850,b000bdezfw,1445.90,http://www.google.com/base/feeds/snippets/1843...,881.00,564.90,564.90
124,adobe captivate 2.0,model- adbcd01561wi vendor- adobe software fea...,adobe captivate 2 software for windows present...,captivate 2 software for windows the captivate...,1,0.626566,0.644523,b000iaolaw,1083.95,http://www.google.com/base/feeds/snippets/2919...,598.95,485.00,485.00
214,avid liquid vs 7 software,video liquid version 7 sftw,avid liquid 7.0 - professional video editing s...,liquid 7.0 - professional video editing softwa...,1,0.922742,0.659447,b000e0v924,888.75,http://www.google.com/base/feeds/snippets/3852...,439.95,448.80,448.80
117,motu digital performer 5 digital audio softwar...,for song writing studio production film scorin...,motu dp5 software music production software,digital performer 5 (dp5) multi-track audio/mi...,1,0.796051,0.784896,b000fiej4k,795.00,http://www.google.com/base/feeds/snippets/1530...,499.95,295.05,295.05
62,hp dss software - ( v. 4.0 ) - complete packag...,- marketing information: hp digital sending so...,hp t1936aa uao digital sending software 4.0 (1...,hp dss 4.0 is the entry-level member of a fami...,1,0.777501,0.764230,b0006nkbn2,630.36,http://www.google.com/base/feeds/snippets/1398...,369.99,260.37,260.37
95,steinberg halion symphonic orchestra virtual i...,halion symphonic orchestra puts the awesome em...,steinberg software halion symphonic orchestra ...,halion symphonic orchestra supports all major ...,1,0.920236,0.631977,b000k6b2l0,374.99,http://www.google.com/base/feeds/snippets/1783...,545.51,-170.52,170.52
234,makemusic finale 2007 win/mac,are you the next mozart richard rodgers john l...,make finale 2007 software music production sof...,finale 2007 - professional notation and score ...,1,0.630493,0.821434,b000h1de00,600.00,http://www.google.com/base/feeds/snippets/1732...,429.95,170.05,170.05
